generate positions, topologies, and systems of the 64 small molecules of interest in vacuum and solvent

In [43]:
import numpy as np
import pickle
import os

In [44]:
# extract smiles in order
freesolv_dataset = np.load(os.path.join(os.getcwd(), 'freesolv_dataset.npy'))
smiles = [i[-1] for i in freesolv_dataset]

In [45]:
len(freesolv_dataset)

64

In [46]:
from perses.utils.openeye import smiles_to_oemol
from perses.utils.openeye import has_undefined_stereocenters
from openforcefield.topology import Molecule
from simtk import unit
import simtk.openmm as openmm
import simtk.openmm.app as app
from openmmforcefields.generators.system_generators import SystemGenerator
temperature = 300 * unit.kelvin

# generate openforcefield.molecule list
dict_of_mols = {_idx : list(val) for _idx, val in enumerate(freesolv_dataset)}
molecules = []
for idx, smi in enumerate([val[-1] for val in dict_of_mols.values()]):
    mol = smiles_to_oemol(smi)
    if not has_undefined_stereocenters(mol):
        dict_of_mols[idx].extend([mol, Molecule.from_openeye(mol)])
        molecules.append(Molecule.from_openeye(mol))
    else:
        print(smi)

#default arguments for SystemGenerators
vacuum_barostat, solvent_barostat = None, openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50)

forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
forcefield_kwargs = {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : None, 'hydrogenMass' : 4 * unit.amus}
nonperiodic_forcefield_kwargs = {'nonbondedMethod': app.NoCutoff}
periodic_forcefield_kwargs = {'nonbondedMethod': app.PME}
small_molecule_forcefield = 'gaff-2.11'

vacuum_system_generator = SystemGenerator(forcefields = forcefield_files, barostat=None, forcefield_kwargs=forcefield_kwargs,nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                         small_molecule_forcefield = small_molecule_forcefield, molecules=molecules, cache=None)
solvent_system_generator = SystemGenerator(forcefields = forcefield_files, barostat=solvent_barostat, forcefield_kwargs=forcefield_kwargs,periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                         small_molecule_forcefield = small_molecule_forcefield, molecules=molecules, cache=None)

INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generati

In [47]:
len(dict_of_mols)

64

In [48]:
from perses.utils.openeye import extractPositionsFromOEMol
from openmoltools.forcefield_generators import generateTopologyFromOEMol
import tqdm
out_pars = {key: [] for key in dict_of_mols.keys()}
for key in list(dict_of_mols.keys()):
    print(key)
    tup = dict_of_mols[key]
    molecule = dict_of_mols[key][-1]
    positions = extractPositionsFromOEMol(tup[5], units = unit.angstroms)
    topology = generateTopologyFromOEMol(tup[5])
    vac_sys = vacuum_system_generator.create_system(topology, molecule)
    out_pars[key].extend([vac_sys, positions, topology])
    
    #solvent
    modeller = app.Modeller(topology, positions)
    modeller.addSolvent(solvent_system_generator.forcefield, model='tip3p', padding=0.9*unit.nanometers, ionicStrength=0.15*unit.molar)
    solvated_topology = modeller.getTopology()
    solvated_positions = modeller.getPositions()
    solvated_positions = unit.quantity.Quantity(value = np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit = unit.nanometers)
    solvent_sys = solvent_system_generator.create_system(solvated_topology, molecule)
    out_pars[key].extend([solvent_sys, solvated_positions, solvated_topology])

INFO:openmmforcefields.generators.template_generators:Requested to generate parameters for residue <Residue 0 (MOL) of chain 0>
INFO:openmmforcefields.generators.template_generators:Generating a residue template for [H]C([H])([H])C([H])([H])C([H])([H])C([H])([H])O[H]
DEBUG:openmmforcefields.generators.template_generators:Total charge is 0
DEBUG:openmmforcefields.generators.template_generators:Using user-provided charges because partial charges are nonzero...
DEBUG:openmmforcefields.generators.template_generators:Generating a conformer...
DEBUG:openmmforcefields.generators.template_generators:Running antechamber...
DEBUG:openmmforcefields.generators.template_generators:antechamber -i in.mdl -fi mdl -o out.mol2 -fo mol2 -s 0 -at 2 -dr no


0


DEBUG:openmmforcefields.generators.template_generators:
Welcome to antechamber 19.0: molecular input file processor.


DEBUG:openmmforcefields.generators.template_generators:parmchk2 -i out.mol2 -f mol2 -p /home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/openmmforcefields/ffxml/amber/gaff/dat/gaff-2.11.dat -o out.frcmod -s %2
DEBUG:openmmforcefields.generators.template_generators:
DEBUG:openmmforcefields.generators.template_generators:Reading GAFF atom types...
DEBUG:openmmforcefields.generators.template_generators:Fixing partial charges...
DEBUG:openmmforcefields.generators.template_generators:[ 0.          0.          0.          0.28       -0.68000001  0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.40000001] e
DEBUG:openmmforcefields.generators.template_generators:[ 0.          0.          0.          0.28       -0.68000001  0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.

1


DEBUG:openmmforcefields.generators.template_generators:
Welcome to antechamber 19.0: molecular input file processor.


DEBUG:openmmforcefields.generators.template_generators:parmchk2 -i out.mol2 -f mol2 -p /home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/openmmforcefields/ffxml/amber/gaff/dat/gaff-2.11.dat -o out.frcmod -s %2
DEBUG:openmmforcefields.generators.template_generators:
DEBUG:openmmforcefields.generators.template_generators:Reading GAFF atom types...
DEBUG:openmmforcefields.generators.template_generators:Fixing partial charges...
DEBUG:openmmforcefields.generators.template_generators:[-0.15000001 -0.15000001 -0.15000001 -0.15000001 -0.15000001  0.0825
  0.66000003 -0.56999999 -0.23250002  0.15000001  0.15000001  0.15000001
  0.15000001  0.15000001  0.06      ] e
DEBUG:openmmforcefields.generators.template_generators:[-0.15000001 -0.15000001 -0.15000001 -0.15000001 -0.15000001  0.0825
  0.66000002 -0.57       -0.23250002  0.15000001  0.15000001  0.15000001
  0.

...

DEBUG:openmmforcefields.generators.template_generators:
Welcome to antechamber 19.0: molecular input file processor.


DEBUG:openmmforcefields.generators.template_generators:parmchk2 -i out.mol2 -f mol2 -p /home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/openmmforcefields/ffxml/amber/gaff/dat/gaff-2.11.dat -o out.frcmod -s %2
DEBUG:openmmforcefields.generators.template_generators:
DEBUG:openmmforcefields.generators.template_generators:Reading GAFF atom types...
DEBUG:openmmforcefields.generators.template_generators:Fixing partial charges...
DEBUG:openmmforcefields.generators.template_generators:[ 0.          0.          0.2402      0.79979998 -0.51999998 -0.51999998
  0.          0.          0.          0.          0.          0.
  0.        ] e
DEBUG:openmmforcefields.generators.template_generators:[ 0.          0.          0.2402      0.79979997 -0.51999998 -0.51999998
  0.          0.          0.          0.          0.          0.
  0.        ] e
DEBUG:openmmforcefie

62


DEBUG:openmmforcefields.generators.template_generators:
Welcome to antechamber 19.0: molecular input file processor.


DEBUG:openmmforcefields.generators.template_generators:parmchk2 -i out.mol2 -f mol2 -p /home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/openmmforcefields/ffxml/amber/gaff/dat/gaff-2.11.dat -o out.frcmod -s %2
DEBUG:openmmforcefields.generators.template_generators:
DEBUG:openmmforcefields.generators.template_generators:Reading GAFF atom types...
DEBUG:openmmforcefields.generators.template_generators:Fixing partial charges...
DEBUG:openmmforcefields.generators.template_generators:[ 0.    0.    0.    0.28  0.28 -0.56  0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.  ] e
DEBUG:openmmforcefields.generators.template_generators:[ 0.    0.    0.    0.28  0.28 -0.56  0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.  ] e
DEBUG:openmmforcefields.generators.template_generators:Creating ffxml contents for additional parameters..

63


DEBUG:openmmforcefields.generators.template_generators:
Welcome to antechamber 19.0: molecular input file processor.


DEBUG:openmmforcefields.generators.template_generators:parmchk2 -i out.mol2 -f mol2 -p /home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/openmmforcefields/ffxml/amber/gaff/dat/gaff-2.11.dat -o out.frcmod -s %2
DEBUG:openmmforcefields.generators.template_generators:
DEBUG:openmmforcefields.generators.template_generators:Reading GAFF atom types...
DEBUG:openmmforcefields.generators.template_generators:Fixing partial charges...
DEBUG:openmmforcefields.generators.template_generators:[ 0.          0.          0.          0.          0.28       -0.68000001
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.40000001] e
DEBUG:openmmforcefields.generators.template_generators:[ 0.          0.          0.          0.          0.28       -0.68000001
  0.          0.          0.          0. 

In [55]:
out_pars[0][1]

Quantity(value=array([[ 0.40389186, -1.0520569 , -0.11219127],
       [-0.5795039 , -2.023816  ,  0.5201428 ],
       [-1.9306709 , -1.9925119 , -0.18846905],
       [-2.9047027 , -2.9950266 ,  0.4140324 ],
       [-4.1327834 , -2.9264126 , -0.29739016],
       [ 0.01529087, -0.02894694, -0.08760782],
       [ 1.3639599 , -1.0678931 ,  0.41353902],
       [ 0.58443373, -1.321838  , -1.1573712 ],
       [-0.16113399, -3.0362406 ,  0.47941652],
       [-0.7075766 , -1.7679754 ,  1.5783566 ],
       [-2.3649855 , -0.9864662 , -0.13419703],
       [-1.8040484 , -2.2064962 , -1.2571932 ],
       [-2.5138862 , -4.0145416 ,  0.33633935],
       [-3.0991368 , -2.7668076 ,  1.4668462 ],
       [-4.729492  , -3.5751565 ,  0.10989294]], dtype=float32), unit=angstrom)

In [ ]:
# import mdtraj as md
# md_top = md.Topology.from_openmm(dict_of_mols[1][4][-1])
# md_traj = md.Trajectory(dict_of_mols[1][4][1].value_in_unit_system(unit.md_unit_system), md_top)
# md_traj.save(f"test.pdb")

In [56]:
#now that we seem to have everything equipped, we can save positions, box_vectors, systems, and topologies
from simtk.openmm import XmlSerializer

for key in tqdm.tqdm(out_pars.keys()):
    vac_sys, vac_pos, vac_top = out_pars[key][:3]
    sol_sys, sol_pos, sol_top = out_pars[key][3:6]
    writer_template = f"lig{format(key, '02')}"
    with open(f"{writer_template}.vacuum.system.xml", 'w') as outfile: #write vac sys
        serialized_system = XmlSerializer.serialize(vac_sys)
        outfile.write(serialized_system)
    with open(f"{writer_template}.solvent.system.xml", 'w') as outfile: #write solvent sys
        serialized_system = XmlSerializer.serialize(sol_sys)
        outfile.write(serialized_system)
    
    np.savez(f"{writer_template}.vacuum.positions.npz", positions = np.array([vac_pos.value_in_unit_system(unit.md_unit_system)]))
    
    box_vecs = sol_sys.getDefaultPeriodicBoxVectors()
    box_vectors = [[box_vecs[0][0]/unit.nanometers, 0., 0.], [0., box_vecs[1][1]/unit.nanometers,0.], [0., 0., box_vecs[2][2]/unit.nanometers]]
    np.savez(f"{writer_template}.solvent.positions.npz", positions = np.array([sol_pos.value_in_unit_system(unit.md_unit_system)]), 
             box_vectors = np.array([box_vectors]))
    
    with open(f"{writer_template}.vacuum.topology.pkl", 'wb') as f:
        pickle.dump(vac_top, f)
    with open(f"{writer_template}.solvent.topology.pkl", 'wb') as f:
        pickle.dump(sol_top, f)
    
        
    
    
    
    

100%|██████████| 64/64 [00:07<00:00,  8.94it/s]


In [ ]:
with open(f"lig18.solvent.system.xml", 'r') as infile:
    state_xml = infile.read()
state = XmlSerializer.deserialize(state_xml)